In [70]:
import requests
import json
import pandas as pd
from tqdm import tqdm
import unicodedata
import time 


In [201]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
}
params = {
    "limit": "40", # limit number of items each page
    "include": "advertisement",
    "aggregation": "2",
    "category": "1846",
    "page": "1", # page number
    "urlkey": "laptop-may-vi-tinh-linh-kien" # product category
}
new_api_request_url = "https://tiki.vn/api/v2/products" # new api from tiki developer
old_api_request_url = "https://tiki.vn/api/personalish/v1/blocks/listings" # current api from tiki webpage

product_id_list = []
df_id = pd.DataFrame([], columns=["id"])
# old api page: 2000 - 209   2000 - 209
# old api no page: 47376 - 988
# new-api-page : 2000 - 198  2000 - 199  2000 - 240 
# new-api-no page:  10080 - 211    8400 - 176

<b> Crawl Product ID <b>

In [202]:
def fetchProductID(headers, params, old_api_request_url):
    df_id_tmp = pd.DataFrame([], columns=["id"])
    pageNum = 1
    running = True
    while running:
        '''With the page number param available, data return only 2000 items with 51 pages 
          If you block this param, the data will fetch until the protocol crash which can up to 1 million items but when you drop duplicates it return only 40 items'''   
        #print("Crawl page number: ", pageNum)
        params["page"] = pageNum
        '''params limit is 40 -> each loop fetch 40 items'''
        # Divided each batch with approximately 2000 items 
        if(len(product_id_list) >= 2000):
            print("Reach 2000 items")
            df_temp = pd.DataFrame(product_id_list, columns=["id"])
            df_id_tmp = pd.concat([df_id_tmp, df_temp], ignore_index=True)
            print(df_id_tmp.shape)
            # Truncate all the list
            del product_id_list[:]
                  
        response = requests.get(old_api_request_url, headers=headers, params=params)

        if response.status_code == 200:
            try:
                if(response.json()['data'] == []):
                    #print(pageNum)
                    break

                for product in response.json()["data"]:
                    id = str(product["id"])
                    product_id_list.append(id)
            except: pass
        else:
            print(f"Error, status code = {response.status_code}")
            pass
        pageNum+=1
    # Fill out the list that not reach 2000 items
    df_temp = pd.DataFrame(product_id_list, columns=["id"])
    df_id_tmp = pd.concat([df_id_tmp, df_temp], ignore_index=True)
    # Truncate all the list
    del product_id_list[:] 
    return df_id_tmp

In [203]:
df_id = fetchProductID(headers, params, old_api_request_url)
# 2000 items - 43 pages - 48 limit
# 2000 items - 51 pages - 40 limit
# 26.4s

Reach 2000 items
(2000, 1)


In [204]:
print(len(product_id_list))
print(df_id.shape) 

0
(2000, 1)


In [232]:
df_id.to_csv('dataset/product_id.csv')

In [205]:
list_temp = df_id["id"].values.tolist()

In [206]:
print(len(list_temp))

2000


<b> Fetch Product Details Data <b>

In [225]:
url = "https://tiki.vn/api/v2/products/{}"
pars = {"platform": "web", 
        "spid": "124939771"
}
product_detail_list = []

In [226]:
def crawlProductData(list_temp, url, headers, pars):
    df_product_tmp = pd.DataFrame([])
    for id in tqdm(list_temp, total=len(list_temp)):
        # Divided each batch with maximum 2000 items 
        if(len(product_detail_list) == 2000):
            print("Reach 2000 items")
            df_temp = pd.DataFrame(product_detail_list)
            df_product = pd.concat([df_product, df_temp], ignore_index=True)
            # Truncate all the list
            del product_detail_list[:]
            print(df_product.shape)

        response = requests.get(url.format(id), headers=headers, params=pars)
        
        if(response.status_code==200):
            try:
                product_detail_list.append(response.json())
                #product = json.loads(response.text)         
                #product_detail_list.append(product)
            except:
                #print("Something else went wrong")
                pass

        else:
            print(f"Error, status code = {response.status_code}")
            pass

        # Fill out the list that not reach 2000 items
        df_temp = pd.DataFrame(product_detail_list)
        df_product_tmp = pd.concat([df_product_tmp, df_temp], ignore_index=True)
        # Truncate all the list
        del product_detail_list[:]
    return df_product_tmp   

In [227]:
df_product = crawlProductData(list_temp, url, headers, pars)
# 10m57s

100%|██████████| 2000/2000 [10:57<00:00,  3.04it/s]


In [249]:
print(len(product_detail_list))
print(len(product_json_list))
print(df_product.shape)
# json.loads() + from_dict(): 1232 items - 10m10s
# no json.loads() + with from_dict(): 1284 items - 10m25s - excel: 1368 - drop: 1339
# no json.loads() + no from_dict(): 1358 items - 10m57s - excel: 1641 - drop: 1479

0
0
(1358, 10)


In [229]:
df_product.head()

,id,master_id,sku,name,url_key,url_path,short_url,type,book_cover,short_description,...,return_and_exchange_policy,is_tier_pricing_available,is_tier_pricing_eligible,asa_cashback_widget,benefits,price_comparison,video_url,best_price_guaranteed,deal_specs,is_inventory_clearance_sale
0,247241481,247241481,1014979908670,Phần Mềm Diệt Virus BKAV Profressional 1 PC 12...,phan-mem-diet-virus-bkav-profressional-1-pc-12...,phan-mem-diet-virus-bkav-profressional-1-pc-12...,https://tiki.vn/product-p247241481.html?spid=1...,configurable,None,Phần Mềm Diệt Virus BKAV Profressional sử dụng...,...,Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 218, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN,NaN,NaN
1,217317583,217317583,3771762210965,Box Orico Đựng Ổ Cứng HDD/SSD 2.5 inch 2520C3 ...,box-orico-dung-o-cung-hdd-ssd-2-5-inch-2520c3-...,box-orico-dung-o-cung-hdd-ssd-2-5-inch-2520c3-...,https://tiki.vn/product-p217317583.html?spid=2...,simple,None,Thương hiệu: OricoDung lượng hỗ trợ tối đa: 4T...,...,Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 218, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN,NaN,NaN
2,74975883,74975883,8856697508143,Bộ Phát Wifi TP-Link Archer C54 Băng Tần Kép C...,bo-phat-wifi-tp-link-archer-c54-bang-tan-kep-c...,bo-phat-wifi-tp-link-archer-c54-bang-tan-kep-c...,https://tiki.vn/product-p74975883.html?spid=74...,simple,None,Bộ Phát Wifi TP-Link Archer C54 Băng Tần Kép C...,...,Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 218, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,"{'sub_title': 'Giá từ 430.000 ₫', 'title': '6 ...",https://sand.tikicdn.com/ts/video/d3/14/01/473...,NaN,NaN,NaN
3,216462048,216462048,5472671799437,Box đựng ổ cứng Orico SSD M2 PCIe NVMe To USB ...,box-dung-o-cung-orico-ssd-m2-pcie-nvme-to-usb-...,box-dung-o-cung-orico-ssd-m2-pcie-nvme-to-usb-...,https://tiki.vn/product-p216462048.html?spid=2...,simple,None,Thương hiệu: OricoDung lượng hỗ trợ: 2TBGiao t...,...,Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 218, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN,NaN,NaN
4,53056161,53056161,3405828458427,Phần mềm Microsoft 365 Family English APAC EM ...,phan-mem-microsoft-365-family-english-apac-em-...,phan-mem-microsoft-365-family-english-apac-em-...,https://tiki.vn/product-p53056161.html?spid=13...,simple,None,Microsoft 365 Family - 6GQ-01144Microsoft 365 ...,...,Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 218, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN,NaN,NaN


In [233]:
df_product.to_csv('dataset/product_full_details.csv', encoding='utf-8-sig', index=False)

In [234]:
df = pd.read_csv('dataset/product_full_details.csv')
df.head()

,id,master_id,sku,name,url_key,url_path,short_url,type,book_cover,short_description,...,return_and_exchange_policy,is_tier_pricing_available,is_tier_pricing_eligible,asa_cashback_widget,benefits,price_comparison,video_url,best_price_guaranteed,deal_specs,is_inventory_clearance_sale
0,247241481,247241481,1014979908670,Phần Mềm Diệt Virus BKAV Profressional 1 PC 12...,phan-mem-diet-virus-bkav-profressional-1-pc-12...,phan-mem-diet-virus-bkav-profressional-1-pc-12...,https://tiki.vn/product-p247241481.html?spid=1...,configurable,NaN,Phần Mềm Diệt Virus BKAV Profressional sử dụng...,...,Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 218, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN,NaN,NaN
1,217317583,217317583,3771762210965,Box Orico Đựng Ổ Cứng HDD/SSD 2.5 inch 2520C3 ...,box-orico-dung-o-cung-hdd-ssd-2-5-inch-2520c3-...,box-orico-dung-o-cung-hdd-ssd-2-5-inch-2520c3-...,https://tiki.vn/product-p217317583.html?spid=2...,simple,NaN,Thương hiệu: OricoDung lượng hỗ trợ tối đa: 4T...,...,Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 218, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN,NaN,NaN
2,74975883,74975883,8856697508143,Bộ Phát Wifi TP-Link Archer C54 Băng Tần Kép C...,bo-phat-wifi-tp-link-archer-c54-bang-tan-kep-c...,bo-phat-wifi-tp-link-archer-c54-bang-tan-kep-c...,https://tiki.vn/product-p74975883.html?spid=74...,simple,NaN,Bộ Phát Wifi TP-Link Archer C54 Băng Tần Kép C...,...,Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 218, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,"{'sub_title': 'Giá từ 430.000 ₫', 'title': '6 ...",https://sand.tikicdn.com/ts/video/d3/14/01/473...,NaN,NaN,NaN
3,216462048,216462048,5472671799437,Box đựng ổ cứng Orico SSD M2 PCIe NVMe To USB ...,box-dung-o-cung-orico-ssd-m2-pcie-nvme-to-usb-...,box-dung-o-cung-orico-ssd-m2-pcie-nvme-to-usb-...,https://tiki.vn/product-p216462048.html?spid=2...,simple,NaN,Thương hiệu: OricoDung lượng hỗ trợ: 2TBGiao t...,...,Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 218, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN,NaN,NaN
4,53056161,53056161,3405828458427,Phần mềm Microsoft 365 Family English APAC EM ...,phan-mem-microsoft-365-family-english-apac-em-...,phan-mem-microsoft-365-family-english-apac-em-...,https://tiki.vn/product-p53056161.html?spid=13...,simple,NaN,Microsoft 365 Family - 6GQ-01144Microsoft 365 ...,...,Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 218, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN,NaN,NaN


In [235]:
df_tmp = df[['current_seller', 'all_time_quantity_sold', 'id', 'brand', 'categories']]
df_tmp.rename(columns={'id': 'product_id'}, inplace=True)
df_tmp.shape # 1358

C:\Users\nthph\AppData\Local\Temp\ipykernel_10888\3633754930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp.rename(columns={'id': 'product_id'}, inplace=True)


(1358, 5)

In [236]:
# Create dim seller
seller = df_tmp['current_seller'].tolist()
seller_list = [eval(p) for p in seller]
df_seller = pd.DataFrame.from_dict(seller_list)
df_seller = df_seller[['id', 'sku', 'name', 'link']]
df_seller.rename(columns={'id': 'seller_id'}, inplace=True)
# fact sale - seller id - product id
df_fact = pd.concat([df_tmp, df_seller], axis=1, ignore_index=False)
df_fact.drop(['sku', 'name', 'link', 'current_seller', 'brand', 'categories'], axis=1, inplace=True)


In [237]:
# Clean dim seller
print(df_seller.shape) # 1358
df_seller_dim = df_seller.drop_duplicates(keep='first')
print(df_seller_dim.shape) # 1358
df_seller_dim.head()

(1358, 4)
(1358, 4)


,seller_id,sku,name,link
0,200265,8496435050154,Kmart Store,https://tiki.vn/cua-hang/kmart-store
1,25203,5453682956104,Lagihitech2,https://tiki.vn/cua-hang/lagihitech2
2,154163,1795795194452,TP-Link Official Store,https://tiki.vn/cua-hang/tp-link-official-store
3,25203,5219119302514,Lagihitech2,https://tiki.vn/cua-hang/lagihitech2
4,198012,6524122468007,Phần mềm Văn phòng,https://tiki.vn/cua-hang/phan-mem-van-phong


In [238]:
# Create dim category
category = df_tmp['categories'].tolist()
category_list = [eval(p) for p in category]
# Tạo url cho subcategory của sp
for item in category_list:
    id = item['id']
    name = item['name']
    # Xử lý chuỗi Tiếng Việt
    name = name.lower()
    name = name.replace("đ", "d").replace(" ", "-")
    name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore')
    name = name.decode('utf-8')
    item['url'] = f"/{name}/c{id}"

df_category = pd.DataFrame.from_dict(category_list)
df_category = df_category[['id','name', 'url']]
df_category.rename(columns={'id': 'category_id'}, inplace=True)
# fact sale - seller_id - product_id - category_id
df_fact = pd.concat([df_fact, df_category], axis=1, ignore_index=False)
df_fact.drop(['name', 'url'], axis=1, inplace=True)


In [239]:
# Clean dim category
print(df_category.shape) # 1358
df_category_dim = df_category.drop_duplicates(keep='first')
print(df_category_dim.shape) # 76
df_category_dim.head()

(1358, 3)
(76, 3)


,category_id,name,url
0,1846,Laptop - Máy Vi Tính - Linh kiện,/laptop---may-vi-tinh---linh-kien/c1846
1,5350,Case Ổ Cứng- Hộp Đựng HDD Box - Dock Ổ Cứng,/case-o-cung--hop-dung-hdd-box---dock-o-cung/c...
2,4293,Router Wifi,/router-wifi/c4293
5,28932,Màn Hình Phổ Thông,/man-hinh-pho-thong/c28932
6,4343,Bộ Kích Sóng Wifi,/bo-kich-song-wifi/c4343


In [240]:
# Create dim brand
brand = df_tmp['brand'].tolist()
brand_list = [eval(p) for p in brand]
df_brand = pd.DataFrame.from_dict(brand_list)
df_brand = df_brand[['id', 'name']]
df_brand.rename(columns={'id': 'brand_id'}, inplace=True)
# fact sale - seller_id - product_id - category_id - brand_id
df_fact = pd.concat([df_fact, df_brand], axis=1, ignore_index=False)
df_fact.drop(['name'], axis=1, inplace=True)


In [241]:
# Clean dim brand
print(df_brand.shape) # 1358
df_brand_dim = df_brand.drop_duplicates(keep='first')
print(df_brand_dim.shape) #  191
df_brand_dim.head()

(1358, 2)
(191, 2)


,brand_id,name
0,143406,Bkav
1,110682,Orico
2,27464,TP-Link
4,19246,Microsoft
5,49067,MSI


In [242]:
# Create dim product 
df_product = df[['id', 'name', 'sku', 'price', 'list_price', 'discount', 'discount_rate', 'inventory_status', 'stock_item']]
df_product.head()

,id,name,sku,price,list_price,discount,discount_rate,inventory_status,stock_item
0,247241481,Phần Mềm Diệt Virus BKAV Profressional 1 PC 12...,1014979908670,190000,190000,0,0,available,"{'max_sale_qty': 1000, 'min_sale_qty': 1, 'pre..."
1,217317583,Box Orico Đựng Ổ Cứng HDD/SSD 2.5 inch 2520C3 ...,3771762210965,97000,100000,3000,3,available,"{'max_sale_qty': 1000, 'min_sale_qty': 1, 'pre..."
2,74975883,Bộ Phát Wifi TP-Link Archer C54 Băng Tần Kép C...,8856697508143,409000,858000,449000,52,available,"{'max_sale_qty': 1000, 'min_sale_qty': 1, 'pre..."
3,216462048,Box đựng ổ cứng Orico SSD M2 PCIe NVMe To USB ...,5472671799437,350000,350000,0,0,available,"{'max_sale_qty': 1000, 'min_sale_qty': 1, 'pre..."
4,53056161,Phần mềm Microsoft 365 Family English APAC EM ...,3405828458427,1345000,1690000,345000,20,available,"{'max_sale_qty': 1000, 'min_sale_qty': 1, 'pre..."


In [243]:
# stock_item: max_sale_qty, qty
stock = df_product['stock_item'].tolist()
stock_list = [eval(p) for p in stock]
df_stock = pd.DataFrame.from_dict(stock_list)
df_stock = df_stock[['max_sale_qty', 'qty']]
# Dim Product
df_product = pd.concat([df_product, df_stock], axis=1, ignore_index=False)
df_product.drop(['stock_item'], axis=1, inplace=True)

In [244]:
# Clean Dim Product
print(df_product.shape) # 1358
df_product_dim = df_product.drop_duplicates(keep='first')
print(df_product_dim.shape) # 1358
df_product_dim.head()

(1358, 10)
(1358, 10)


,id,name,sku,price,list_price,discount,discount_rate,inventory_status,max_sale_qty,qty
0,247241481,Phần Mềm Diệt Virus BKAV Profressional 1 PC 12...,1014979908670,190000,190000,0,0,available,1000,1000
1,217317583,Box Orico Đựng Ổ Cứng HDD/SSD 2.5 inch 2520C3 ...,3771762210965,97000,100000,3000,3,available,1000,1000
2,74975883,Bộ Phát Wifi TP-Link Archer C54 Băng Tần Kép C...,8856697508143,409000,858000,449000,52,available,1000,1000
3,216462048,Box đựng ổ cứng Orico SSD M2 PCIe NVMe To USB ...,5472671799437,350000,350000,0,0,available,1000,1000
4,53056161,Phần mềm Microsoft 365 Family English APAC EM ...,3405828458427,1345000,1690000,345000,20,available,1000,1000


In [245]:
print(df_fact.shape) # 1358
df_fact.head()

(1358, 5)


,all_time_quantity_sold,product_id,seller_id,category_id,brand_id
0,1308.0,247241481,200265,1846,143406
1,258.0,217317583,25203,5350,110682
2,4334.0,74975883,154163,4293,27464
3,145.0,216462048,25203,5350,110682
4,562.0,53056161,198012,1846,19246


In [247]:
df_fact.to_csv('dataset/fact_sale.csv', index=False)
df_product_dim.to_csv('dataset/dim_product.csv', encoding='utf-8-sig', index=False)
df_brand_dim.to_csv('dataset/dim_brand.csv', encoding='utf-8-sig', index=False)
df_seller_dim.to_csv('dataset/dim_seller.csv', encoding='utf-8-sig', index=False)
df_category_dim.to_csv('dataset/dim_category.csv', encoding='utf-8-sig', index=False)

In [ ]:
# import to MYSQL